In [4]:
import pandas as pd
import numpy as np
import os
import operator
from nltk.probability import FreqDist, MLEProbDist
import math
import collections 
import os

In [6]:
# VALUES OF K
kvals = list(range(4,10))

In [5]:
class OrderedCounter(collections.Counter, collections.OrderedDict):
    def __repr__(self):
        return "%s(%r)" % (self.__class__.__name__, collections.OrderedDict(self))

In [7]:
def read_fasta_file(fn):
    
    data = []
    with open(fn, 'r') as fh:
        lines = []
        for line in fh:
            if line[0] != '>':
                lines.append(line.rstrip())
            else:
                data.append(''.join(lines))
                lines = []
        data.append(''.join(lines))
    
    del data[0]
    return data


In [8]:
def save_results_to_file(data, name, outputFolder):
    
    currPath = os.getcwd()
    os.chdir(outputFolder)
    
    if os.path.exists(name):
        os.remove(name)
    
    with open(name, 'w') as f:
        for item in data:
            f.write("%s\n" % item)
    
    os.chdir(currPath)
    return

In [9]:
def count_kmers(read, k,counts):
    # Calculate how many kmers of length k there are
    num_kmers = len(read) - k + 1
    # Loop over the kmer start positions
    for i in range(num_kmers):
        # Slice the string to get the kmer
        kmer = read[i:i+k]
        # Add the kmer to the dictionary if it's not there
        if kmer not in counts:
            counts[kmer] = 0
        # Increment the count for this kmer
        counts[kmer] += 1
    # Return the final counts
    return counts


In [10]:
def sieve(n):
    # Creating the array of prime numbers of k-mer
    primes=[]
    chkthis = 2
    while len(primes) < n:
        ptest    = [chkthis for i in primes if chkthis%i == 0]
        primes  += [] if ptest else [chkthis]
        chkthis += 1
    
    primes = np.log(primes)
    return primes

In [11]:
def  godel(read,out_array,godel_numbers):
    # Godel numbers in a given read.
    # For every string
    for elem in read.keys():
        point_in_string=0
        # If the key is new
        if elem not in godel_numbers:
            godel_numbers[elem] = 0
        # Calculate the godel number for the key
        for x in elem:
            if x == 'A' or x == 'a':
                godel_numbers[elem]+=out_array[point_in_string]*1
                point_in_string=point_in_string+1
            if x == 'T' or x == 't':
                godel_numbers[elem]+=out_array[point_in_string]*4
                point_in_string=point_in_string+1
            if x == 'G' or x == 'g':
                godel_numbers[elem]+=out_array[point_in_string]*3
                point_in_string=point_in_string+1
            if x == 'C' or x == 'c':
                godel_numbers[elem]+=out_array[point_in_string]*2
                point_in_string=point_in_string+1
    
    return godel_numbers

In [12]:
# Creating the .txt
files = [f for f in os.listdir('input/') if f.endswith('.fasta')]
file = files[0]
filename = file[0:-6]
filename_txt = filename + '.txt'

if not os.path.exists('input/'+filename_txt):
    data = read_fasta_file('input/' + file)
    save_results_to_file(data, filename_txt, 'input')
    del data



In [13]:
for k in kvals:
    print("k = {0}".format(k))
    txt_filehandle = open('input/' + filename_txt, "r")
    # Start with an empty dictionary
    counts = {}
    for row in txt_filehandle:
        counts = count_kmers(row.rstrip(),k,counts)

    #Close input file
    txt_filehandle.close

    # Sort dictionary by value
    sorted_counts = dict(sorted(counts.items(), key=operator.itemgetter(1)))
    sorted_data=np.array(list(sorted_counts.values()))
    #Array of the dictionary keys
    sorted_keys=np.array(list(sorted_counts.keys()))
    # Count of the class values
    counter=OrderedCounter(sorted_data)
    # Count of the frequency of each class
    counter_class=OrderedCounter(counter.keys())
    # Array of the classes
    classes={}
    classes=[count for n,count in counter.items() for i in range(count)]

    #Calculate entropy with nltk library
    freq_dist = FreqDist(sorted_counts)
    prob_dist = MLEProbDist(freq_dist)
    px = [prob_dist.prob(x) for x,n_x in sorted_counts.items()]
    e_x = [-p_x*math.log(p_x,2) for p_x in px]

    # Calculate Godel Numbers
    prime_numbers=sieve(k)
    godel_numbers={}
    godel_numbers=godel(sorted_counts,prime_numbers,godel_numbers)
    del prime_numbers
    
    if not os.path.exists('results'):
        os.makedirs('results')

    # Create info file
    info = open("results/info_k_{0}.txt".format(k),"w")
    df_data = pd.DataFrame({'K-mer':sorted_keys,'Value':sorted_data,'Godel_number':np.array(list(godel_numbers.values())),'Entropy':e_x}) 
    info.write(df_data.to_string())
    info.close()


k = 4
k = 5
k = 6
k = 7
k = 8
k = 9
